In [1]:
#层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。我们先来看一个简单的例子：创建一个Series，并用一个由列表或数组组成的列表作为索引：

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],[1, 2, 3, 1, 3, 1, 2, 2, 3]]
                       )

In [4]:
data

a  1    0.266641
   2   -0.385255
   3    0.259041
b  1    1.253039
   3    0.266730
c  1   -0.131164
   2   -0.702335
d  2   -0.386655
   3    0.008003
dtype: float64

In [5]:
#看到的结果是经过美化的带有MultiIndex索引的Series的格式。索引之间的“间隔”表示“直接使用上面的标签”：

In [6]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
#对于一个层次化索引的对象，可以使用所谓的部分索引，使用它选取数据子集的操作更简单：

In [8]:
data['b']

1    1.253039
3    0.266730
dtype: float64

In [9]:
data['b':'c']

b  1    1.253039
   3    0.266730
c  1   -0.131164
   2   -0.702335
dtype: float64

In [10]:
data.loc[['b', 'd']]

b  1    1.253039
   3    0.266730
d  2   -0.386655
   3    0.008003
dtype: float64

In [11]:
#有时甚至还可以在“内层”中进行选取：

In [12]:
data.loc[:, 2]
#外标签都抓取，看内层的标签

a   -0.385255
c   -0.702335
d   -0.386655
dtype: float64

In [13]:
#层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。例如，可以通过unstack方法将这段数据重新安排到一个DataFrame中：
data.unstack()

,1,2,3
a,0.266641,-0.385255,0.259041
b,1.253039,NaN,0.266730
c,-0.131164,-0.702335,NaN
d,NaN,-0.386655,0.008003


In [14]:
#unstack的逆运算是stack：
data.unstack().stack()

a  1    0.266641
   2   -0.385255
   3    0.259041
b  1    1.253039
   3    0.266730
c  1   -0.131164
   2   -0.702335
d  2   -0.386655
   3    0.008003
dtype: float64

In [15]:
#对于一个DataFrame，每条轴都可以有分层索引：
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
       index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=[['Ohio', 'Ohio', 'Colorado'],
                ['Green', 'Red', 'Green']])

In [16]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [17]:
#各层都可以有名字（可以是字符串，也可以是别的Python对象）。如果指定了名称，它们就会显示在控制台输出中：

In [18]:
frame.index.names = ['key1','key2']

In [19]:
frame.columns.names = ['state','color']

In [20]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [21]:
#有了部分列索引，因此可以轻松选取列分组：
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [22]:
#可以单独创建MultiIndex然后复用。上面那个DataFrame中的（带有分级名称）列可以这样创建：
#pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],names=['state', 'color'])

# 重排与分级排序

In [24]:
#有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）：
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [25]:
#而sort_index则根据单个级别中的值对数据进行排序。交换级别时，常常也会用到sort_index，这样最终结果就是按照指定顺序进行字母排序了：
frame.sort_index(level=1)
#level=1表示针对列的内标签进行排序

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [26]:
frame.swaplevel(0, 1).sort_index(level=0)
#swaplevel(0, 1)就是key1，key2交换，level=0表示针对列的外标签进行排序

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

# 根据级别汇总统计

In [27]:
#许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于指定在某条轴上求和的级别。再以上面那个DataFrame为例，我们可以根据行或列上的级别来进行求和：

In [28]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [29]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [30]:
frame.sum(level='color', axis=1)
#axis=1按行进行求和操作

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

# 使用DataFrame的列进行索引

In [31]:
#人们经常想要将DataFrame的一个或多个列当做行索引来用，或者可能希望将行索引变成DataFrame的列。以下面这个DataFrame为例：

In [32]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                       'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                       'd': [0, 1, 2, 0, 1, 2, 3]})

In [33]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [34]:
#DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame：
frame2 = frame.set_index(['c','d'])

In [35]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [36]:
#默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来：
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [37]:
#reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面：
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
